In [28]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leinad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle
from tensorflow import keras

In [46]:
with open("intents.json") as file:
    data = json.load(file)

In [47]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [48]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [49]:
try:
	model.load("model.tflearn")
except:
	model = tflearn.DNN(net)
	model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
	model.save("model.tflearn")

Training Step: 4999  | time: 0.018s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 32/34
Training Step: 5000  | time: 0.025s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 34/34
--
INFO:tensorflow:c:\Users\Leinad\Documents\Tech_Villain\TechandHi\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [50]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [51]:
def chat():
	print("Start Talking with the bot(type quit to stop!")
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = numpy.argmax(results)
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in data["intents"]:
				if tg['tag'] == tag:
					responses = tg['responses']	
			print(random.choice(responses))
			print("\n")
		else:
			print("Sorry you feel this way\nMind me recommending a place that can provide you with the care you nend?")


In [52]:
chat()

Start Talking with the bot(type quit to stop!
I'm really sorry to hear that you're feeling this way. Remember, you're stronger than you think. Tough times don't last forever, and you have the power to overcome them. Believe in yourself and take things one step at a time.
It's completely normal to feel overwhelmed at times. Just remember, you have an incredible strength within you. You've faced challenges before and come out on top. You can do it again.
I'm here to support you through this. You're not alone. Sometimes, the darkest moments can lead to the most beautiful transformations. Keep moving forward, and you'll find your light.


Sorry you feel this way
Mind me recommending a place that can provide you with the care you nend?
Take care. If you ever need assistance or someone to talk to, please feel free to reach out. You're not alone in this.


Glad I could help and thank you for trusting me to open up as per the privacy regulations your data is secured
but I'd still like to recom